### Plot earthquakes in the Philippines using <font color=fucshia>_matplotlib_</font>, <font color=green>_csv_</font>, and <font color=orange>_cartopy_</font> in Python
__We won't use <font color=navy>Pandas</font> and <font color=purple>Geopandas</font> here so we get familiarized with the csv library__ (and keep our environment small :D)

Before we start importing the necessary packages, let's first download our __list of earthquakes__ from CSEM EMSC via their 'Search earthquakes' function. 
1. Go to this link https://www.emsc-csem.org/Earthquake/?filter=yes
2. In the 'Region Name' form, type 'Philippines'  and select all.
3. In the 'Felt Earthquakes' form, tick the box and set the 'Min' and 'Max' to I to VIII. 
4. Download the list as csv.

Now, you need to __install Miniconda or Anaconda__. Basically, it's a package manager which lets you easily download any Python package/library. I recommend installing a package via `conda-forge` instead of `conda only` like `conda install -c conda-forge geopandas` instead of `conda install geopandas`. 

Here are the __libraries and packages__ you need to install. For those new to Python, use Anaconda instead of Miniconda the former has a GUI while the latter uses a command line. Basically, Anaconda lets you just click the packages you want to install. It took me weeks to figure out how mini/Anaconda works, how to properly install packages, etc. 

1. cartopy https://anaconda.org/conda-forge/cartopy for geometry operations and basemap function
2. matplotlib https://anaconda.org/conda-forge/matplotlib for plotting the map
3. natsort https://anaconda.org/anaconda/natsort to sort your files alphabetically
4. tqdm https://anaconda.org/conda-forge/tqdm for the progress bar while the photos are being exported
5. opencv https://anaconda.org/conda-forge/opencv to compile the maps and convert to video


__Import packages__ and set working directory. You can set the working directory using `os.chdir(r'folder_path')` because sometimes the script doesn't work because some files cannot be found by Python.

In [1]:
import os
import csv
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
from cartopy.feature import ShapelyFeature
from cartopy.io.shapereader import Reader
from datetime import datetime 
from tqdm import tqdm
from natsort import humansorted
# os.chdir(r'C:\Users\imper\Documents\new_GISfiles\earthquakesPH')

Add this line in your Jupyter Notebooks to make sure your __Matplotlib plots are shown inside the notebook.__

In [2]:
%matplotlib inline 

Set the bounds as a list. A bounding box is formed using the __coordinates of the lower left point and upper right points__. In this list, the format is `[latitude lleft, latitude uright, longitude  lleft, longitude uright]`;

set the basemap as Stamen's `terrain-background` map https://scitools.org.uk/cartopy/docs/latest/gallery/eyja_volcano.html; 

read the fault lines shapefile;

and finally, set the folder where you will save all the pictures of the map for each earthquake. 

In [11]:
bounds = [116.9283371, 126.90534668, 4.58693981, 21.07014084]
stamen_terrain = cimgt.Stamen('terrain-background')
fault_line = ShapelyFeature(Reader('./faultLines.shp').geometries(), 
                            ccrs.epsg(32651), linewidth=1, 
                            edgecolor='black', facecolor='None')
save_path = r'.//'  
date,time,lon,lat,mag = [],[],[],[],[]

Read the earthquakes csv. We use `next()` to skip the first row which are the column titles. Make sure to __check first the csv__ before importing to make sure you are setting the __correct delimiter__. Take note that I renamed the column titles of the columns/field I'll use to a shorter name in lowercase.

The `sorted` function sorts the csv by the `date` then `time`. You can set the range to 5 so it will only produce 5 maps/pictures if you are still testing the code. 

In [ ]:
with open ('./export_EMSC_Jul2011-Apr2021.csv') as file:
    reader = csv.reader(file, delimiter=';') 
    next(reader)
    date_sorted = sorted(reader, key=lambda col: datetime.strptime
                         (col[0] + ' ' + col[1], '%Y-%m-%d %H:%M:%S'))
    for col in date_sorted:
        date.append(col[0])
        time.append(str(col[1]))
        lat.append(int(float(col[2])))
        lon.append(int(float(col[3])))
        mag.append(int(float(col[7])))
    index = range(len(date_sorted))
#     index = range(10)

To get a map of each earthquake, we need to use a `for loop()` to __iterate/read each row in the csv__, extract all the needed information, and make the map, and save it in a folder.

And while the loop is running, we will also run a __progress bar__ which says which row is being iterated out of the total rows. We put the loop inside `tdqm()` and update the progress bar for each row that is iterated/read.

`with tqdm(index) as pbar:
   for loop()
   for each row iterated:
       update the progress bar`

Now we will make our `for loop()`. The __first line__ assigns an index to each of the column we will use in the csv. __Lines 2-7__ will set up the map, __line 8__ will set the title for each map based on the row's/earthquake's date and time, __line 9__ will plot the earthquake based on the coordinates, and __line 10__ will add the fault line above the basemap but below the marker.

__Line 20__ will save each map in your disk and name the file based on its index so that when we convert the pictures to a video, the maps are compiled in proper order. You can comment out Line 19 and un-comment line 20 to __just show the map instead of saving__ each one.

In [ ]:
for id,d,t,x,y,m,i in zip(tqdm(range(len(index))),date,time,lon,lat,mag,index):
    fig = plt.figure(figsize=(15,10), dpi=250) 
    ax  = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree()) 
    d = datetime.strptime(d, '%Y-%m-%d')
    d = d.strftime('%Y %b %d')
    ax.set_extent(bounds) 
#     ax.add_image(stamen_terrain, 8)
    plt.suptitle('Earthquakes in the Philippines from 2011 to 2021' + 
                 '\n' +'with active fault lines' + 
                 '\n' + str(d) + ' --- ' + str(t) +
                 '\n' + 'Magnitude: ' + str(m) +
                 ' --- ' + str(i+1) + '/' + str(len(index)),
                 ha='left', x=0.363, y=0.97)
    ax.plot(x, y, 'ro', markersize=5) 
    ax.add_feature(fault_line, zorder=1)
    fault_line_legend = mlines.Line2D([1], [1], color='black')
    ax.legend([fault_line_legend], ['Fault lines'], loc='lower left', fancybox='True')
    plt.savefig(save_path+'eq{}'.format(i+1))
#     plt.show()    

Compile all the maps and __convert to a video__ using imageio.

In [4]:
filenames = humansorted([fn for fn in os.listdir('.') if fn.endswith('.png')])
fps = 2             # Desired, "real" frames per second
fps_vlc = 30        # Frames per second needed for proper playback in VLC
with imageio.get_writer('earthquake_video.mp4', fps=fps_vlc) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        for i in range(fps_vlc // fps):
            writer.append_data(image)
    writer.close()

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (3750, 2500) to (3760, 2512) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


That's it! This is a simple visualization I made when I was starting in Python. Putting this in a Jupyter Notebook made it easier to explain the process. You can download the repository on github and modify the code to do further analyses like showing which areas are affected in each earthquake or leave a transluscent mark for each earthquake to make a density map showing which areas frequently experience an earthquake.

This Notebook is for anyone to use.  :D

-- [Miguel Imperial](#https://www.linkedin.com/in/miguel-imperial-8abaa31a0/)